In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from glob import glob
from shapely.geometry import LineString, Point
import folium
import matplotlib.pyplot as plt
import pytz
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
import SDS_slope

In [2]:
transects = gpd.read_file("transects_extended.geojson").set_index("id")
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0581-0000,nzd0581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68763 -43.62938, 172.68393 -43..."
nzd0581-0001,nzd0581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68829 -43.62992, 172.68459 -43..."
nzd0581-0002,nzd0581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68895 -43.63046, 172.68525 -43..."


In [3]:
new_transects = transects[transects.index.str.startswith("nzd")]# & (transects.index > "nzd0562") & transects.beach_slope.isna()]
new_transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.110,0.0681,0.2000,-0.174010,196.0,192.0,0.003917,17.659820,458.983904,21.423910,325.937325,None,"LINESTRING (172.96367 -34.42911, 172.96351 -34..."
nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,0.115,0.0738,0.2000,-0.181775,196.0,193.0,0.005612,15.338270,353.476544,18.800972,328.697573,None,"LINESTRING (172.96457 -34.4291, 172.9644 -34.4..."
nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.120,0.0770,0.2000,-0.306998,196.0,193.0,0.019757,14.242079,282.327917,16.802616,334.822195,None,"LINESTRING (172.96547 -34.42908, 172.9653 -34...."
nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,0.065,0.0546,0.0759,-0.344997,196.0,194.0,0.032463,12.443181,217.289816,14.740754,344.022130,None,"LINESTRING (172.96637 -34.42907, 172.9662 -34...."
nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.065,0.0568,0.0788,-0.418404,196.0,194.0,0.052214,11.504372,194.647497,13.951613,356.546817,None,"LINESTRING (172.96727 -34.42906, 172.9671 -34...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0581-0000,nzd0581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68763 -43.62938, 172.68393 -43..."
nzd0581-0001,nzd0581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68829 -43.62992, 172.68459 -43..."
nzd0581-0002,nzd0581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68895 -43.63046, 172.68525 -43..."


In [4]:
plt.ioff()
def estimate_slopes(site_id):
  df = pd.read_csv(f"data/{site_id}/transect_time_series.csv")
  df.index = pd.to_datetime(df.dates)
  df.drop(columns=["dates", "satname"], inplace=True)
  tides = pd.read_csv(f"data/{site_id}/tides.csv")
  tides.dates = pd.to_datetime(tides.dates)
  tides.set_index("dates", inplace=True)
  assert all(pd.to_datetime(df.index).round("10min") == tides.index)
  # slope estimation settings
  days_in_year = 365.2425
  seconds_in_day = 24*3600
  settings_slope = {'slope_min':        0.01,                  # minimum slope to trial
                    'slope_max':        0.2,                    # maximum slope to trial
                    'delta_slope':      0.005,                  # slope increment
                    'date_range':       [1999,2020],            # range of dates over which to perform the analysis
                    'n_days':           8,                      # sampling period [days]
                    'n0':               50,                     # parameter for Nyquist criterium in Lomb-Scargle transforms
                    'freqs_cutoff':     1./(seconds_in_day*30), # 1 month frequency
                    'delta_f':          100*1e-10,              # deltaf for identifying peak tidal frequency band
                    'prc_conf':         0.05,                   # percentage above minimum to define confidence bands in energy curve
                    }
  settings_slope['date_range'] = [pytz.utc.localize(datetime(settings_slope['date_range'][0],5,1)),
                                  pytz.utc.localize(datetime(settings_slope['date_range'][1],1,1))]
  beach_slopes = SDS_slope.range_slopes(settings_slope['slope_min'], settings_slope['slope_max'], settings_slope['delta_slope'])

  # find tidal peak frequency (can choose 7 or 8 in this case)
  settings_slope['n_days'] = 7
  settings_slope['freqs_max'] = SDS_slope.find_tide_peak(df.index,tides.tide,settings_slope)
  # estimate beach-face slopes along the transects
  slope_est, cis = dict([]), dict([])
  for key in df.keys():
      # remove NaNs
      idx_nan = np.isnan(df[key])
      dates = [df.index[_] for _ in np.where(~idx_nan)[0]]
      tide = tides.tide.to_numpy()[~idx_nan]
      composite = df[key][~idx_nan]
      # apply tidal correction
      tsall = SDS_slope.tide_correct(composite,tide,beach_slopes)
      title = 'Transect %s'%key
      #SDS_slope.plot_spectrum_all(dates,composite,tsall,settings_slope, title)
      slope_est[key],cis[key] = SDS_slope.integrate_power_spectrum(dates,tsall,settings_slope)
      #print('Beach slope at transect %s: %.3f'%(key, slope_est[key]))
  return pd.DataFrame({"beach_slope": slope_est,
                       "cil": {k: v[0] for k,v in cis.items()},
                       "ciu": {k: v[1] for k,v in cis.items()},
                      }, index=slope_est.keys())

new_slopes = process_map(estimate_slopes, new_transects.site_id.unique())
#estimate_slopes("nzd0563")

  0%|          | 0/580 [00:00<?, ?it/s]

/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


...

/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


/mnt/CoastSat/SDS_slope.py:710: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


In [5]:
(pd.concat(new_slopes).beach_slope - transects.beach_slope).describe()

count    32654.000000
mean        -0.000011
std          0.000616
min         -0.020000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.010000
Name: beach_slope, dtype: float64

In [6]:
np.rad2deg(np.arctan(.01))

0.5729386976834859

In [7]:
transects.update(pd.concat(new_slopes))
transects.to_file("transects_extended.geojson")